## Studying NER

In [1]:
import pandas as pd
import numpy as np
import sklearn
from ner import ner
from presidio_analyzer import AnalyzerEngine, BatchAnalyzerEngine, RecognizerRegistry


In [65]:
import spacy
import spacy_transformers
spacy.load('en_core_web_lg')


In [15]:
df = pd.read_csv("dataset/table_order.csv")
pd.pandas.set_option('display.max_columns', None)
df

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,10100,S18_1749,30,136.00,3,2003-01-06,2003-01-13,2003-01-10,Shipped,NaN,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216.0,114200,*,Classic Cars,1:10,Welly Diecast Productions,*,4823.0,103.125817,213.098062
1,10100,S18_2248,50,55.09,2,2003-01-06,2003-01-13,2003-01-10,Shipped,NaN,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216.0,114200,*,Vintage Cars,1:24,Red Start Diecast,*,2728.0,15.949021,33.802497
2,10100,S18_4409,22,75.46,4,2003-01-06,2003-01-13,2003-01-10,Shipped,NaN,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216.0,114200,*,Vintage Cars,1:32,Studio M Art Models,*,2685.0,21.882745,43.659252
3,10100,S24_3969,49,35.29,1,2003-01-06,2003-01-13,2003-01-10,Shipped,NaN,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216.0,114200,*,Vintage Cars,1:32,Autoart Studio Design,*,550.0,16.061205,35.118038
4,10101,S18_2325,25,108.06,4,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,NaN,Frankfurt,NaN,60528,Germany,1504.0,59700,*,Trucks and Buses,1:32,Autoart Studio Design,*,24.0,25.660152,46.427639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991,10425,S24_2300,49,127.79,9,2005-05-31,2005-06-07,NaN,In Process,NaN,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200,*,Vintage Cars,1:18,Welly Diecast Productions,*,6763.0,71.973408,117.440000
2992,10425,S24_2840,31,31.82,5,2005-05-31,2005-06-07,NaN,In Process,NaN,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200,*,Classic Cars,1:18,Gearbox Collectibles,*,9771.0,94.808700,152.250990
2993,10425,S32_1268,41,83.79,11,2005-05-31,2005-06-07,NaN,In Process,NaN,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200,*,Classic Cars,1:18,Min Lin Diecast,*,9762.0,69.061199,138.994773
2994,10425,S32_2509,11,50.32,6,2005-05-31,2005-06-07,NaN,In Process,NaN,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200,*,Classic Cars,1:18,Red Start Diecast,*,8909.0,93.836843,208.542691


In [56]:
df = df.sample(n=min(1000, df.shape[0])) 


In [291]:
analyzer = AnalyzerEngine(supported_languages=["en", "fr"])
batch_analyzer = BatchAnalyzerEngine(analyzer_engine=analyzer)

In [292]:
analyzer_results = list(batch_analyzer.analyze_dict(df.to_dict(orient="list"), language="en"))

In [293]:
analyzer_results

[DictAnalyzerResult(key='orderNumber', value=[10373, 10157, 10245, 10190, 10225, 10425, 10181, 10105, 10421, 10225, 10359, 10367, 10391, 10285, 10119, 10178, 10370, 10198, 10211, 10285, 10416, 10250, 10392, 10200, 10394, 10405, 10173, 10179, 10310, 10249, 10173, 10364, 10165, 10136, 10163, 10273, 10117, 10142, 10300, 10251, 10280, 10276, 10250, 10167, 10261, 10185, 10318, 10374, 10331, 10310], recognizer_results=[[], [type: DATE_TIME, start: 0, end: 5, score: 0.85], [], [], [type: DATE_TIME, start: 0, end: 5, score: 0.85], [], [], [], [], [type: DATE_TIME, start: 0, end: 5, score: 0.85], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [type: DATE_TIME, start: 0, end: 5, score: 0.85], [], [], [], [type: DATE_TIME, start: 0, end: 5, score: 0.85], [], [], [], [], [], [], [type: DATE_TIME, start: 0, end: 5, score: 0.85], [], [], [], [type: DATE_TIME, start: 0, end: 5, score: 0.85], [], [], [], [], [], [type: DATE_TIME, start: 0, end: 5, score: 0.85], [], []]),
 DictAnalyzer

In [294]:
#questo ciclo controlla per ogni colonna, se il suo tipo è object. Se è object, allora aggiunge al dizionario l 
#un'entrata la cui chiave sarà il nome della colonna e il cui valore è il type del primo elemento delle varie liste 
#presenti in recognizer_results solo se queste liste non sono vuote
l = {}
for w, j in zip(df.columns, analyzer_results):
        if df[w].dtype == 'object':
            l[w] = [i[0].entity_type for i in j.recognizer_results if len(i) > 0]

In [295]:
list_entities = {}
cols = [col for col in df.columns]

In [296]:
for col in cols:
        list_entities[col] = None

In [297]:
#creo lista con solo le chiavi (colonne) la cui lista di valori contiene 
#un numero di elementi maggiore di 0.1 * il numero di rows nel campione
cols = [col for col in l.keys() if len(l[col]) > 0.1 * df.shape[0]]
cols

['orderDate',
 'requiredDate',
 'shippedDate',
 'comments',
 'customerName',
 'contactLastName',
 'contactFirstName',
 'phone',
 'addressLine1',
 'addressLine2',
 'city',
 'state',
 'postalCode',
 'country',
 'productVendor']

In [298]:
for col in cols:
    lst = [i for i in l[col]]
    if ('LOCATION' in lst) and ('name' not in col.lower()) and (
            len([i for i in l[col] if i == 'LOCATION']) > 0.1 * df.shape[0]):
            #il secondo elemento è una sorta di probabilità/confidenza che quella colonna sia di quel determinato tipo
        list_entities[col] = ['LOCATION', len([i for i in l[col] if i == 'LOCATION'])/df.shape[0]]
    else:
        most_freq = max(set(lst), key=lst.count)
        list_entities[col] = [most_freq, len([i for i in lst if i == most_freq])/df.shape[0]]

In [299]:
loc_cols = [[i, list_entities[i][0]] for i in list_entities.keys() if list_entities[i] is not None]
loc_cols

[['orderDate', 'DATE_TIME'],
 ['requiredDate', 'DATE_TIME'],
 ['shippedDate', 'DATE_TIME'],
 ['comments', 'LOCATION'],
 ['customerName', 'PERSON'],
 ['contactLastName', 'PERSON'],
 ['contactFirstName', 'PERSON'],
 ['phone', 'PHONE_NUMBER'],
 ['addressLine1', 'PERSON'],
 ['addressLine2', 'LOCATION'],
 ['city', 'LOCATION'],
 ['state', 'LOCATION'],
 ['postalCode', 'DATE_TIME'],
 ['country', 'LOCATION'],
 ['productVendor', 'LOCATION']]

In [300]:
loc_cols = [i[0] for i in loc_cols if i[1] == 'LOCATION']
loc_cols

['comments', 'addressLine2', 'city', 'state', 'country', 'productVendor']

In [301]:
addresses = ['street', 'rue', 'via', 'square', 'avenue', 'place', 'strada']
for col in loc_cols:
    list_addresses = [i for i in df[col].fillna('none') if any(ele in i.lower() for ele in addresses)]
    if len(list_addresses) > 0.1 * int(df.shape[0]):
        list_entities[col] = ['ADDRESS', len(list_addresses)/int(df.shape[0])]


In [302]:
list_addresses

[]

In [303]:
list_entities #è un dizionario chiave/colonna: [entity_type, confidence]

{'orderNumber': None,
 'productCode': None,
 'quantityOrdered': None,
 'priceEach': None,
 'orderLineNumber': None,
 'orderDate': ['DATE_TIME', 1.0],
 'requiredDate': ['DATE_TIME', 1.0],
 'shippedDate': ['DATE_TIME', 0.94],
 'status': None,
 'comments': ['LOCATION', 0.78],
 'customerNumber': None,
 'customerName': ['PERSON', 0.12],
 'contactLastName': ['PERSON', 0.44],
 'contactFirstName': ['PERSON', 0.82],
 'phone': ['PHONE_NUMBER', 0.54],
 'addressLine1': ['PERSON', 0.18],
 'addressLine2': ['LOCATION', 0.82],
 'city': ['LOCATION', 0.76],
 'state': ['LOCATION', 0.72],
 'postalCode': ['DATE_TIME', 0.16],
 'country': ['LOCATION', 1.0],
 'salesRepEmployeeNumber': None,
 'creditLimit': None,
 'productName': None,
 'productLine': None,
 'productScale': None,
 'productVendor': ['LOCATION', 0.1],
 'productDescription': None,
 'quantityInStock': None,
 'buyPrice': None,
 'MSRP': None}

## Trying to improve ner function

In [2]:
from typing import List
import pprint

from presidio_analyzer import AnalyzerEngine, PatternRecognizer, EntityRecognizer, Pattern, RecognizerResult
from presidio_analyzer.recognizer_registry import RecognizerRegistry
from presidio_analyzer.nlp_engine import NlpEngine, SpacyNlpEngine, NlpArtifacts, NlpEngineProvider
from presidio_analyzer.context_aware_enhancers import LemmaContextAwareEnhancer

In [3]:
ner("dataset/table_order.csv")

Starting...


{'orderNumber': None,
 'productCode': None,
 'quantityOrdered': None,
 'priceEach': None,
 'orderLineNumber': None,
 'orderDate': ['DATE_TIME', 1.0],
 'requiredDate': ['DATE_TIME', 1.0],
 'shippedDate': ['DATE_TIME', 0.948],
 'status': None,
 'comments': None,
 'customerNumber': None,
 'customerName': ['PERSON', 0.071],
 'contactLastName': ['PERSON', 0.502],
 'contactFirstName': ['PERSON', 0.823],
 'phone': ['PHONE_NUMBER', 0.594],
 'addressLine1': ['ADDRESS', 0.554],
 'addressLine2': None,
 'city': ['LOCATION', 0.79],
 'state': ['LOCATION', 0.156],
 'postalCode': None,
 'country': ['LOCATION', 1.0],
 'salesRepEmployeeNumber': None,
 'creditLimit': None,
 'productName': None,
 'productLine': None,
 'productScale': None,
 'productVendor': ['PERSON', 0.138],
 'productDescription': None,
 'quantityInStock': None,
 'buyPrice': None,
 'MSRP': None}

In [12]:
df = pd.read_csv("dataset/table_order.csv")


In [16]:
df

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,10100,S18_1749,30,136.00,3,2003-01-06,2003-01-13,2003-01-10,Shipped,NaN,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216.0,114200,*,Classic Cars,1:10,Welly Diecast Productions,*,4823.0,103.125817,213.098062
1,10100,S18_2248,50,55.09,2,2003-01-06,2003-01-13,2003-01-10,Shipped,NaN,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216.0,114200,*,Vintage Cars,1:24,Red Start Diecast,*,2728.0,15.949021,33.802497
2,10100,S18_4409,22,75.46,4,2003-01-06,2003-01-13,2003-01-10,Shipped,NaN,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216.0,114200,*,Vintage Cars,1:32,Studio M Art Models,*,2685.0,21.882745,43.659252
3,10100,S24_3969,49,35.29,1,2003-01-06,2003-01-13,2003-01-10,Shipped,NaN,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216.0,114200,*,Vintage Cars,1:32,Autoart Studio Design,*,550.0,16.061205,35.118038
4,10101,S18_2325,25,108.06,4,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,NaN,Frankfurt,NaN,60528,Germany,1504.0,59700,*,Trucks and Buses,1:32,Autoart Studio Design,*,24.0,25.660152,46.427639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991,10425,S24_2300,49,127.79,9,2005-05-31,2005-06-07,NaN,In Process,NaN,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200,*,Vintage Cars,1:18,Welly Diecast Productions,*,6763.0,71.973408,117.440000
2992,10425,S24_2840,31,31.82,5,2005-05-31,2005-06-07,NaN,In Process,NaN,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200,*,Classic Cars,1:18,Gearbox Collectibles,*,9771.0,94.808700,152.250990
2993,10425,S32_1268,41,83.79,11,2005-05-31,2005-06-07,NaN,In Process,NaN,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200,*,Classic Cars,1:18,Min Lin Diecast,*,9762.0,69.061199,138.994773
2994,10425,S32_2509,11,50.32,6,2005-05-31,2005-06-07,NaN,In Process,NaN,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200,*,Classic Cars,1:18,Red Start Diecast,*,8909.0,93.836843,208.542691


In [59]:
df = df.sample(n=min(1000, df.shape[0]))

In [60]:
df = df.fillna(0)

In [61]:
addresses = ['Street', 'Rue', 'Via', 'Square', 'Avenue', 'Place', 'Strada', 'St', 'Lane', 
    'Road', 'Boulevard', 'Ln', "Rd", "Highway" "Drive", "Av", "Hwy", "Blvd", "Corso", "Piazza", 
    "Calle", "Plaza", "Avenida", "Rambla", "C/"]

In [62]:
addresses_recognizer = PatternRecognizer(supported_entity="ADDRESS", deny_list=addresses)


In [63]:
#using a roberta-based language model to improve performance
'''configuration = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "en", "model_name": "en_core_web_trf"}],
}

# Create NLP engine based on configuration
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine_robertaBased = provider.create_engine()

# Pass the created NLP engine and supported_languages to the AnalyzerEngine
analyzer = AnalyzerEngine(
    nlp_engine = nlp_engine_robertaBased, 
    supported_languages=["en", "fr", "es", "it"]
) '''

'configuration = {\n    "nlp_engine_name": "spacy",\n    "models": [{"lang_code": "en", "model_name": "en_core_web_trf"}],\n}\n\n# Create NLP engine based on configuration\nprovider = NlpEngineProvider(nlp_configuration=configuration)\nnlp_engine_robertaBased = provider.create_engine()\n\n# Pass the created NLP engine and supported_languages to the AnalyzerEngine\nanalyzer = AnalyzerEngine(\n    nlp_engine = nlp_engine_robertaBased, \n    supported_languages=["en", "fr", "es", "it"]\n) '

In [64]:
analyzer = AnalyzerEngine(supported_languages=["en", "fr"])
analyzer.registry.add_recognizer(addresses_recognizer)
batch_analyzer = BatchAnalyzerEngine(analyzer_engine=analyzer)

In [65]:
analyzer_results = list(batch_analyzer.analyze_dict(df.to_dict(orient="list"), language="en"))

In [66]:
analyzer_results

[DictAnalyzerResult(key='orderNumber', value=[10299, 10149, 10105, 10219, 10360, 10321, 10216, 10360, 10198, 10356, 10404, 10173, 10264, 10142, 10280, 10371, 10191, 10259, 10350, 10339, 10101, 10108, 10104, 10278, 10416, 10195, 10192, 10133, 10373, 10222, 10222, 10212, 10333, 10299, 10390, 10276, 10142, 10307, 10304, 10297, 10220, 10229, 10350, 10238, 10297, 10353, 10159, 10261, 10106, 10143, 10321, 10107, 10316, 10238, 10234, 10204, 10293, 10359, 10192, 10181, 10155, 10382, 10321, 10347, 10147, 10331, 10159, 10281, 10194, 10131, 10420, 10165, 10164, 10321, 10310, 10159, 10397, 10225, 10139, 10372, 10284, 10296, 10131, 10398, 10297, 10235, 10130, 10250, 10374, 10165, 10122, 10167, 10416, 10252, 10194, 10342, 10205, 10305, 10197, 10258, 10389, 10370, 10124, 10142, 10355, 10190, 10208, 10418, 10331, 10208, 10189, 10411, 10330, 10241, 10368, 10176, 10283, 10260, 10401, 10317, 10253, 10398, 10167, 10260, 10203, 10370, 10143, 10378, 10134, 10145, 10207, 10354, 10197, 10268, 10375, 10377, 10

In [67]:
l = {}
for w, j in zip(df.columns, analyzer_results):
    if df[w].dtype == 'object':
        l[w] = [i[0].entity_type for i in j.recognizer_results if len(i) > 0]

In [68]:
list_entities = {}
cols = [col for col in df.columns]

In [69]:
for col in cols:
    list_entities[col] = None

In [70]:
cols = [col for col in l.keys() if len(l[col]) > 0.1 * df.shape[0]]

In [71]:
for col in cols:
    lst = [i for i in l[col]]
    if ('LOCATION' in lst) and ('name' not in col.lower()) and (
            len([i for i in l[col] if i == 'LOCATION']) > 0.1 * df.shape[0]):
        list_entities[col] = ['LOCATION', len([i for i in l[col] if i == 'LOCATION'])/df.shape[0]]
    elif (('postal' in col.lower()) and ('code' in col.lower())) or (('zip' in col.lower()) and ('code' in col.lower())) or (('zip' in col.lower())): 
        list_entities[col] = ['ZIPCODE', len([i for i in l[col]])/df.shape[0]]
    else:
        most_freq = max(set(lst), key=lst.count)
        list_entities[col] = [most_freq, len([i for i in lst if i == most_freq])/df.shape[0]]

In [73]:
list_entities

{'orderNumber': None,
 'productCode': None,
 'quantityOrdered': None,
 'priceEach': None,
 'orderLineNumber': None,
 'orderDate': ['DATE_TIME', 1.0],
 'requiredDate': ['DATE_TIME', 1.0],
 'shippedDate': ['DATE_TIME', 0.956],
 'status': None,
 'comments': None,
 'customerNumber': None,
 'customerName': ['PERSON', 0.068],
 'contactLastName': ['PERSON', 0.517],
 'contactFirstName': ['PERSON', 0.807],
 'phone': ['PHONE_NUMBER', 0.572],
 'addressLine1': ['ADDRESS', 0.55],
 'addressLine2': None,
 'city': ['LOCATION', 0.787],
 'state': ['LOCATION', 0.147],
 'postalCode': ['ZIPCODE', 0.105],
 'country': ['LOCATION', 1.0],
 'salesRepEmployeeNumber': None,
 'creditLimit': None,
 'productName': None,
 'productLine': None,
 'productScale': None,
 'productVendor': ['PERSON', 0.148],
 'productDescription': None,
 'quantityInStock': None,
 'buyPrice': None,
 'MSRP': None}

## Test ner on other datasets

#### telco_customer dataset

In [4]:
df_ibm = pd.read_csv("dataset/Telco_customer_churn.csv")


In [5]:
df_ibm

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,1,United States,California,Landers,92285,"34.341737, -116.539416",34.341737,-116.539416,Female,...,Two year,Yes,Bank transfer (automatic),21.15,1419.4,No,0,45,5306,NaN
7039,6840-RESVB,1,United States,California,Adelanto,92301,"34.667815, -117.536183",34.667815,-117.536183,Male,...,One year,Yes,Mailed check,84.80,1990.5,No,0,59,2140,NaN
7040,2234-XADUH,1,United States,California,Amboy,92304,"34.559882, -115.637164",34.559882,-115.637164,Female,...,One year,Yes,Credit card (automatic),103.20,7362.9,No,0,71,5560,NaN
7041,4801-JZAZL,1,United States,California,Angelus Oaks,92305,"34.1678, -116.86433",34.167800,-116.864330,Female,...,Month-to-month,Yes,Electronic check,29.60,346.45,No,0,59,2793,NaN


In [6]:
df_ibm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

In [19]:
ner("dataset/Telco_customer_churn.csv")

Starting...


{'CustomerID': None,
 'Count': None,
 'Country': ['LOCATION', 1.0],
 'State': ['LOCATION', 1.0],
 'City': ['LOCATION', 0.531],
 'Zip Code': None,
 'Lat Long': ['US_DRIVER_LICENSE', 0.936],
 'Latitude': None,
 'Longitude': None,
 'Gender': None,
 'Senior Citizen': None,
 'Partner': None,
 'Dependents': None,
 'Tenure Months': None,
 'Phone Service': None,
 'Multiple Lines': None,
 'Internet Service': None,
 'Online Security': None,
 'Online Backup': None,
 'Device Protection': None,
 'Tech Support': None,
 'Streaming TV': None,
 'Streaming Movies': None,
 'Contract': ['DATE_TIME', 1.0],
 'Paperless Billing': None,
 'Payment Method': None,
 'Monthly Charges': None,
 'Total Charges': None,
 'Churn Label': None,
 'Churn Value': None,
 'Churn Score': None,
 'CLTV': None,
 'Churn Reason': None}

### List of real usa addresses

In [18]:
df_USAddresses = pd.read_csv("dataset/list_of_real_usa_addresses.csv")


In [9]:
df_USAddresses[0:50]

,address,city,state,zip
0,777 Brockton Avenue,Abington,MA,2351
1,30 Memorial Drive,Avon,MA,2322
2,250 Hartford Avenue,Bellingham,MA,2019
3,700 Oak Street,Brockton,MA,2301
4,66-4 Parkhurst Rd,Chelmsford,MA,1824
5,591 Memorial Dr,Chicopee,MA,1020
6,55 Brooksby Village Way,Danvers,MA,1923
7,137 Teaticket Hwy,East Falmouth,MA,2536
8,42 Fairhaven Commons Way,Fairhaven,MA,2719
9,374 William S Canning Blvd,Fall River,MA,2721


In [10]:
df_USAddresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   address  234 non-null    object
 1   city     234 non-null    object
 2   state    234 non-null    object
 3   zip      234 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 7.4+ KB


In [11]:
ner("list_of_real_usa_addresses.csv")

Starting...


{'address': ['ADDRESS', 0.6153846153846154],
 'city': ['LOCATION', 0.5384615384615384],
 'state': ['LOCATION', 0.6794871794871795],
 'zip': None}